In [1]:
import sqlite3 as sql

In [5]:
conn = sql.connect('Chinook.db')

In [6]:
cursor = conn.cursor()

In [92]:
# this allows to select all columns with a limit of 5 records
cursor.execute('SELECT * FROM albums LIMIT 5')

In [25]:
# fetches all records
cursor.fetchall()

# fethcmany() has the default parameter value of 1
# cursor.fetchmany(5)

[]

In [24]:
cursor.execute('INSERT INTO artists(ArtistID, Name) values(276, "Kiyo")')

In [26]:
# commits the changes to the database
conn.commit()

In [41]:
albums = [(348,'Okay Lang Yan',276), 
          (349, 'Eba', 276), 
          (350, 'Nandito Na', 276),
          (351, 'Dantay', 276),
          (352, 'Ikaw Lang', 276)
         ]

In [42]:
cursor.executemany('INSERT INTO albums(AlbumID, Title, ArtistID) values(?, ?, ?)', albums)

In [43]:
conn.commit()

In [39]:
cursor.execute('DELETE FROM albums WHERE ArtistID=276')

In [40]:
conn.commit()

In [65]:
cursor.execute('SELECT * FROM artists')

In [66]:
list = cursor.fetchmany(5)

In [67]:
for items in list:
    print(items[0], ' - ', items[1])

1  -  AC/DC
2  -  Accept
3  -  Aerosmith
4  -  Alanis Morissette
5  -  Alice In Chains


In [45]:
# pandas is a library that allows us to analyze data by creating models and 
# fetching information from the database, query, or file
import pandas as pd

In [47]:
df_albums = pd.read_sql_query('SELECT * FROM albums', conn)

In [48]:
df_albums

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
347,348,Okay Lang Yan,276
348,349,Eba,276
349,350,Nandito Na,276
350,351,Dantay,276


In [49]:
df_albums.shape

(352, 3)

In [50]:
df_albums.to_csv('Albums.csv', index=False)

In [84]:
# ON query is best used if the column name are different then opt to use USING if they are of the same
# ON simply repeats the column name and its records thus not the best thing to use if they are of the same name
df_test1 = pd.read_sql_query('SELECT * FROM albums INNER JOIN artists ON albums.ArtistId = artists.ArtistId', conn)
df_test1

,AlbumId,Title,ArtistId,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,1,AC/DC
1,2,Balls to the Wall,2,2,Accept
2,3,Restless and Wild,2,2,Accept
3,4,Let There Be Rock,1,1,AC/DC
4,5,Big Ones,3,3,Aerosmith
...,...,...,...,...,...
347,348,Okay Lang Yan,276,276,Kiyo
348,349,Eba,276,276,Kiyo
349,350,Nandito Na,276,276,Kiyo
350,351,Dantay,276,276,Kiyo


In [80]:
df_test2 = pd.read_sql_query('SELECT * FROM albums CROSS JOIN artists', conn)
df_test2.shape[0]

97152

In [81]:
# USING query allows us to use the name of column from two tables as means to inner join them
# lessens the redundancy by allowing column name of the same name to join bind two tables together
df_test3 = pd.read_sql_query('SELECT * FROM albums INNER JOIN artists USING (ArtistId)', conn)
df_test3

,AlbumId,Title,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith
...,...,...,...,...
347,348,Okay Lang Yan,276,Kiyo
348,349,Eba,276,Kiyo
349,350,Nandito Na,276,Kiyo
350,351,Dantay,276,Kiyo


In [85]:
# Natural Join is simply INNER JOIN that uses USING query
df_test4 = pd.read_sql_query('SELECT * FROM albums NATURAL JOIN artists', conn)
df_test4

,AlbumId,Title,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith
...,...,...,...,...
347,348,Okay Lang Yan,276,Kiyo
348,349,Eba,276,Kiyo
349,350,Nandito Na,276,Kiyo
350,351,Dantay,276,Kiyo


In [90]:
df_test5 = pd.read_sql_query('SELECT * FROM albums LEFT OUTER JOIN artists USING(ArtistId) UNION SELECT * FROM artists LEFT OUTER JOIN albums USING(ArtistId)', conn)
df_test5

,AlbumId,Title,ArtistId,Name
0,1,AC/DC,1.0,For Those About To Rock We Salute You
1,1,AC/DC,4.0,Let There Be Rock
2,1,For Those About To Rock We Salute You,1.0,AC/DC
3,2,Accept,2.0,Balls to the Wall
4,2,Accept,3.0,Restless and Wild
...,...,...,...,...
770,348,Okay Lang Yan,276.0,Kiyo
771,349,Eba,276.0,Kiyo
772,350,Nandito Na,276.0,Kiyo
773,351,Dantay,276.0,Kiyo


In [91]:
df_test5.to_csv('TEST1.csv', index=False)